<a href="https://colab.research.google.com/github/mricos/bird2vec/blob/master/config/wavegan_tf112_py36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training Wavegan with tensorflow 1.12.0 and python 3.6.7

Official implementation of WaveGAN, a machine learning algorithm which learns to generate raw audio waveforms.

https://github.com/chrisdonahue/wavegan/

@inproceedings {donahue2019wavegan,
  title={Adversarial Audio Synthesis},
  author={Donahue, Chris and McAuley, Julian and Puckette, Miller},
  booktitle={ICLR},
  year={2019}
}

### Mount Google Drive
  In this notebook we mount our google drive so that we can store the output of the model while it trains.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Install recommended dependencies
    pip install tensorflow-gpu==1.12.0
    pip install scipy==1.0.0
    pip install matplotlib==3.0.2
    pip install librosa==0.6.2
    
    It's worth it to note that this code maybe compatible different versions of TensorFlow, however we follow the recommendations of the author.

In [0]:
!pip install tensorflow-gpu==1.12.0
!pip install scipy==1.0.0
!pip install matplotlib==3.0.2
!pip install librosa==0.6.2

    100% |████████████████████████████████| 281.7MB 78kB/s 
    100% |████████████████████████████████| 3.1MB 8.5MB/s 
tensorflow 1.13.1 has requirement tensorboard<1.14.0,>=1.13.0, but you'll have tensorboard 1.12.2 which is incompatible.
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
    100% |████████████████████████████████| 50.0MB 928kB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
cvxpy 1.0.15 has requirement scipy>=1.1.0, but you'll have scipy 1.0.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: scipy 1.1.0
    Uninstalling scipy-1.1.0:
      Successfully uninstalled scipy-1.1.0


    100% |████████████████████████████████| 12.9MB 3.3MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: matplotlib 3.0.3
    Uninstalling matplotlib-3.0.3:
      Successfully uninstalled matplotlib-3.0.3


    100% |████████████████████████████████| 1.6MB 17.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
Successfully built librosa
  Found existing installation: librosa 0.6.3
    Uninstalling librosa-0.6.3:
      Successfully uninstalled librosa-0.6.3


## Install cuda9.0 for compatability

Here we download the CUDA toolkit from the Nvidia site, unpack it, add an apt-key to install the CUDA 9 libraires

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo dpkg -i /content/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
print('add key: ok')
!sudo apt-get update
!sudo apt-get install cuda-libraries-9-0

--2019-04-02 23:16:31--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.182.215
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.182.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?tqxAFDQW8D-J-sa8oZz1PSaXAuCF5NCceTs6qZH7tDj1n-AUGeZlecXsfqoo0cGpV5AndMlm9NwsMd48v9NOp1fPlakZQXALzDfqAZUcIJUsFQbDSYQ2OorqXtcZmsNNkMLhTGmHlbIBuxdo13IJB6H-8El3ub1xH-xAWfXIDao2P-vSO-FyXF-mKQGnPgqaP4XTV95JIiY-bMDiw2h- [following]
--2019-04-02 23:16:32--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?tqxAFDQW8D-J-sa8oZz1PSaXAuCF5NCceTs6qZH7tDj1n-AUGeZlecXsfqoo0cGpV5AndMlm9NwsMd48v9NOp1fPlakZQXALzDfqAZUcIJUsFQ

## Clone wavegan, download unzip SC09 data

One of the main points of this notebook book is to train a model without having the data beforehand. In the next cell, we clone the repository that contains the model, download the data we want to train on and unzip the files in place to its own folder.

In [0]:
!git clone https://github.com/chrisdonahue/wavegan.git

Cloning into 'wavegan'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 74.27 MiB | 26.19 MiB/s, done.
Resolving deltas: 100% (325/325), done.
--2019-04-02 23:25:42--  http://deepyeti.ucsd.edu/cdonahue/wavegan/data/sc09.tar.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 541227122 (516M) [application/octet-stream]
Saving to: ‘sc09.tar.gz’

sc09.tar.gz         100%[===================>] 516.15M  11.2MB/s    in 47s     

2019-04-02 23:26:29 (11.0 MB/s) - ‘sc09.tar.gz’ saved [541227122/541227122]



In [0]:
#!wget http://deepyeti.ucsd.edu/cdonahue/wavegan/data/sc09.tar.gz #about 60 seconds
#!tar -xzf /content/sc09.tar.gz #in /content
!mkdir /content/bird2vec
!mkdir /content/bird2vec/train

!cp /content/gdrive/My\ Drive/files/wav/bird2vec/train/* \
   /content/bird2vec/train/

cp: cannot stat '/content/gdrive/My Drive/files/wav/bird2vec/train/*': No such file or directory


Before moving forward, we want to confirm that our cuda9 install is infact working. 
       - Note: At this point, the colab instance has both CUDA 9 and CUDA 10          installed. You can confirm this by opening the file explorer to the left. Following this path:
        ('>', Files> [^],>usr>local)
        The CUDA directories should be present. In any event, if CUDA 9 did not install properly, the next cell will throw an error.

In [0]:
# Confirm GPU is running
from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
if len(get_available_gpus()) == 0:
    for i in range(4):
        print('WARNING: Not running on a GPU! See above for faster generation')
else:
    print('Training on GPU')

Training on GPU


### Start training

In the next cell the code to train the model will be executed. A few things to note:
        - The model is training directly from the '/content/' folder.
        - The output of the model will be stored in the afore linked google drive at ''/content/gdrive/My Drive/wgtrain.
        - The data is being read from '/content/SC09/train'
        
Since the data consists of small bits of audio, it is necessary to add additional flags:
        - 'data_first_slice' to signal small clips
        - 'data_fast_wav' to skip parsing with librosa and use scipy.
        
WARNING: If the data is a collection of MP3's do not use these flags
        
 Other flags:
        - 'wavegan_genr_pp' adds a post-processing filter to reduce noise
        - 'wavegan_dim' decreases the model size (to allow for faster training)
        - 'data_slice_rate' increase the length of the sample the model generates (16384 == 1 second, 65536 == 4 seconds)
        - 'train_summary_secs' increase the gap between checkpoints
        


In [0]:
!export CUDA_VISIBLE_DEVICES="0"
!python /content/wavegan/train_wavegan.py train ./gdrive/My\ Drive/wgtrain --data_dir /content/sc09/train/ --data_first_slice --data_fast_wav --data_slice_len 16384 --wavegan_dim 64 --wavegan_genr_pp --train_summary_secs 300

Found 18620 audio files in specified directory
--------------------------------------------------------------------------------
Generator vars
[100, 16384] (1638400): G/z_project/dense/kernel:0
[16384] (16384): G/z_project/dense/bias:0
[1, 25, 512, 1024] (13107200): G/upconv_0/conv2d_transpose/kernel:0
[512] (512): G/upconv_0/conv2d_transpose/bias:0
[1, 25, 256, 512] (3276800): G/upconv_1/conv2d_transpose/kernel:0
[256] (256): G/upconv_1/conv2d_transpose/bias:0
[1, 25, 128, 256] (819200): G/upconv_2/conv2d_transpose/kernel:0
[128] (128): G/upconv_2/conv2d_transpose/bias:0
[1, 25, 64, 128] (204800): G/upconv_3/conv2d_transpose/kernel:0
[64] (64): G/upconv_3/conv2d_transpose/bias:0
[1, 25, 1, 64] (1600): G/upconv_4/conv2d_transpose/kernel:0
[1] (1): G/upconv_4/conv2d_transpose/bias:0
[512, 1, 1] (512): G/pp_filt/conv1d/kernel:0
Total params: 19065857 (72.73 MB)
--------------------------------------------------------------------------------
Discriminator vars
[25, 1, 64] (1600): D/downco

The model is training. Please see the Tensorboard notebook for instructions on monitoring the training process